In [54]:
# import all the necessary libraries

import numpy as np
import pandas as pd
from datetime import datetime as dt
import itertools
from collections import defaultdict

# %matplotlib inline

In [55]:
# Read the data from a csv to a Pandas dataframe

data_loc = "C:\\Users\\matt\\jupyter-notebooks\\predicting-soccer\\data\\"

raw_data = []

raw_data.append(pd.read_csv(data_loc + '0607-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '0708-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '0809-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '0910-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1011-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1112-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1213-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1314-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1415-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1516-E0.csv'))
raw_data.append(pd.read_csv(data_loc + '1617-E0.csv'))


In [56]:
# Parse date string data as time date format

def parse_date(date):
    if date == '':
        return None
    else:
        return dt.strptime(date, "%d/%m/%y").date()
            
for year in raw_data:
    year.Date = year.Date.apply(parse_date)

In [57]:
# Filter the data points related to the gameplay

filtered_data = []
columns_req = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR']

for year in raw_data:
    filtered_data.append(year[columns_req])

# filtered_data[0]

In [58]:
# # Optimise the column naming structure for grouping using pandas groupby method

# for i, season in enumerate(filtered_data):
#     filtered_data[i].columns = [[    '', 'Home', 'Away', 'Home', 'Away',    ''],
#                                 ['Date', 'Team', 'Team',  'FTG',  'FTG', 'FTR']]

# filtered_data[0].head(n=5)

In [60]:
def goals_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []

    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']
        HTGC = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)

    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    matchWeeks = int( len(playing_stat) / len(teams) * 2 ) + 1
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1, matchWeeks)]).T
    GoalsConceded[0] = 0

    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]

    return GoalsConceded


def get_gss(playing_stat):
    GC = get_goals_conceded(playing_stat)

    j = 0
    HTGC = []
    ATGC = []
    
    matchRounds = len(playing_stat) 
    
    for i in range(matchRounds):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
#         HTGS.append(GS.loc[ht][j])
#         ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
        
#     playing_stat['HTGS'] = HTGS
#     playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC
    
    return playing_stat


# Apply to the filtered dataset
for i, season in enumerate(filtered_data):
    GC = get_gss(filtered_data[i])
    print(GC)
    

c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


           Date          HomeTeam          AwayTeam  FTHG  FTAG FTR  HTGC  \
0    2006-08-19           Arsenal       Aston Villa     1     1   D     0   
1    2006-08-19            Bolton         Tottenham     2     0   H     0   
2    2006-08-19           Everton           Watford     2     1   H     0   
3    2006-08-19         Newcastle             Wigan     2     1   H     0   
4    2006-08-19        Portsmouth         Blackburn     3     0   H     0   
5    2006-08-19           Reading     Middlesbrough     3     2   H     0   
6    2006-08-19  Sheffield United         Liverpool     1     1   D     0   
7    2006-08-19          West Ham          Charlton     3     1   H     0   
8    2006-08-20           Chelsea          Man City     3     0   H     0   
9    2006-08-20        Man United            Fulham     5     1   H     0   
10   2006-08-22         Tottenham  Sheffield United     2     0   H     2   
11   2006-08-22           Watford          West Ham     1     1   D     2   

           Date       HomeTeam       AwayTeam  FTHG  FTAG FTR  HTGC  ATGC
0    2008-08-16        Arsenal      West Brom     1     0   H     0     0
1    2008-08-16         Bolton          Stoke     3     1   H     0     0
2    2008-08-16        Everton      Blackburn     2     3   A     0     0
3    2008-08-16           Hull         Fulham     2     1   H     0     0
4    2008-08-16  Middlesbrough      Tottenham     2     1   H     0     0
5    2008-08-16     Sunderland      Liverpool     0     1   A     0     0
6    2008-08-16       West Ham          Wigan     2     1   H     0     0
7    2008-08-17    Aston Villa       Man City     4     2   H     0     0
8    2008-08-17        Chelsea     Portsmouth     4     0   H     0     0
9    2008-08-17     Man United      Newcastle     1     1   D     0     0
10   2008-08-23      Blackburn           Hull     1     1   D     2     1
11   2008-08-23         Fulham        Arsenal     1     0   H     2     0
12   2008-08-23      Liverpool  Middle

           Date     HomeTeam     AwayTeam  FTHG  FTAG FTR  HTGC  ATGC
0    2010-08-14  Aston Villa     West Ham     3     0   H     0     0
1    2010-08-14    Blackburn      Everton     1     0   H     0     0
2    2010-08-14       Bolton       Fulham     0     0   D     0     0
3    2010-08-14      Chelsea    West Brom     6     0   H     0     0
4    2010-08-14   Sunderland   Birmingham     2     2   D     0     0
5    2010-08-14    Tottenham     Man City     0     0   D     0     0
6    2010-08-14        Wigan    Blackpool     0     4   A     0     0
7    2010-08-14       Wolves        Stoke     2     1   H     0     0
8    2010-08-15    Liverpool      Arsenal     1     1   D     0     0
9    2010-08-16   Man United    Newcastle     3     0   H     0     0
10   2010-08-21      Arsenal    Blackpool     6     0   H     1     0
11   2010-08-21   Birmingham    Blackburn     2     1   H     2     0
12   2010-08-21      Everton       Wolves     1     1   D     1     1
13   2010-08-21     

           Date     HomeTeam     AwayTeam  FTHG  FTAG FTR  HTGC  ATGC
0    2012-08-18      Arsenal   Sunderland     0     0   D     0     0
1    2012-08-18       Fulham      Norwich     5     0   H     0     0
2    2012-08-18    Newcastle    Tottenham     2     1   H     0     0
3    2012-08-18          QPR      Swansea     0     5   A     0     0
4    2012-08-18      Reading        Stoke     1     1   D     0     0
5    2012-08-18    West Brom    Liverpool     3     0   H     0     0
6    2012-08-18     West Ham  Aston Villa     1     0   H     0     0
7    2012-08-19     Man City  Southampton     3     2   H     0     0
8    2012-08-19        Wigan      Chelsea     0     2   A     0     0
9    2012-08-20      Everton   Man United     1     0   H     0     0
10   2012-08-22      Chelsea      Reading     4     2   H     0     1
11   2012-08-25  Aston Villa      Everton     1     3   A     1     0
12   2012-08-25      Chelsea    Newcastle     2     0   H     0     1
13   2012-08-25   Ma

           Date        HomeTeam        AwayTeam  FTHG  FTAG FTR  HTGC  ATGC
0    2014-08-16         Arsenal  Crystal Palace     2     1   H     0     0
1    2014-08-16       Leicester         Everton     2     2   D     0     0
2    2014-08-16      Man United         Swansea     1     2   A     0     0
3    2014-08-16             QPR            Hull     0     1   A     0     0
4    2014-08-16           Stoke     Aston Villa     0     1   A     0     0
5    2014-08-16       West Brom      Sunderland     2     2   D     0     0
6    2014-08-16        West Ham       Tottenham     0     1   A     0     0
7    2014-08-17       Liverpool     Southampton     2     1   H     0     0
8    2014-08-17       Newcastle        Man City     0     2   A     0     0
9    2014-08-18         Burnley         Chelsea     1     3   A     0     0
10   2014-08-23     Aston Villa       Newcastle     0     0   D     0     2
11   2014-08-23         Chelsea       Leicester     2     0   H     1     2
12   2014-08

           Date        HomeTeam        AwayTeam  FTHG  FTAG FTR  HTGC  ATGC
0    2016-08-13         Burnley         Swansea     0     1   A     0     0
1    2016-08-13  Crystal Palace       West Brom     0     1   A     0     0
2    2016-08-13         Everton       Tottenham     1     1   D     0     0
3    2016-08-13            Hull       Leicester     2     1   H     0     0
4    2016-08-13        Man City      Sunderland     2     1   H     0     0
5    2016-08-13   Middlesbrough           Stoke     1     1   D     0     0
6    2016-08-13     Southampton         Watford     1     1   D     0     0
7    2016-08-14         Arsenal       Liverpool     3     4   A     0     0
8    2016-08-14     Bournemouth      Man United     1     3   A     0     0
9    2016-08-15         Chelsea        West Ham     2     1   H     0     0
10   2016-08-19      Man United     Southampton     2     0   H     1     1
11   2016-08-20         Burnley       Liverpool     2     0   H     1     3
12   2016-08

In [23]:
# #*** GOALS CONCEDED ****   
# # Gets the goals conceded agg arranged by teams and matchweek

def get_goals_conceded(playing_stat):
    # Create a dictionary with team names as keys
    teams = {}
    for i in playing_stat.groupby('HomeTeam').mean().T.columns:
        teams[i] = []
    
    # the value corresponding to keys is a list containing the match location.
    for i in range(len(playing_stat)):
        ATGC = playing_stat.iloc[i]['FTHG']
        HTGC = playing_stat.iloc[i]['FTAG']
        teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
        teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
    # Create a dataframe for goals scored where rows are teams and cols are matchweek.
    GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
    GoalsConceded[0] = 0
    # Aggregate to get uptil that point
    for i in range(2,39):
        GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
    return GoalsConceded


def get_gss(playing_stat):
    GC = get_goals_conceded(playing_stat)

    j = 0
    HTGC = []
    ATGC = []

    for i in range(380):
        ht = playing_stat.iloc[i].HomeTeam
        at = playing_stat.iloc[i].AwayTeam
        HTGS.append(GS.loc[ht][j])
        ATGS.append(GS.loc[at][j])
        HTGC.append(GC.loc[ht][j])
        ATGC.append(GC.loc[at][j])
        
        if ((i + 1)% 10) == 0:
            j = j + 1
        
    playing_stat['HTGS'] = HTGS
    playing_stat['ATGS'] = ATGS
    playing_stat['HTGC'] = HTGC
    playing_stat['ATGC'] = ATGC
    
    return playing_stat

get_gss(filtered_data[0])
filtered_data[0]

# # Apply to the filtered dataset
# for i, season in enumerate(filtered_data):
#     get_gss(filtered_data[i])


KeyError: 'HomeTeam'

In [21]:
# Add the cumulative goals scored

def add_cumulative_score_field(frame, cum_field):
    #  Input: 'frame' is a data frame input that has multiple teams to group.
    #         They are grouped by the 'cum_field' string. Teams are either playing home or away.
    # Output: This function has a null return. But it modifies the input frame.
    #         This works as the input frame is a byRef variable asssignment.

    frame_stacked = frame.stack(0)
    total_pts = frame_stacked.groupby('Team')[cum_field].cumsum()

    # Insert the new cumulative sum values in to the frame with the correctly named columns.
    col_heading = "t_scored_{}".format(cum_field)
    frame[('Home', col_heading)] = total_pts[:, 'Home']
    frame[('Away', col_heading)] = total_pts[:, 'Away']
    
    
# Add the cumulative Full Time Scored Goals for both teams.
for i, season in enumerate(filtered_data):
    add_cumulative_score_field(filtered_data[i], 'FTG')

# filtered_data[0]


c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
c:\users\matt\envs\jupyter-notebook\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [172]:
# #*** GOALS SCORED ***
# # Gets the goals scored agg arranged by teams and matchweek
# def get_goals_scored(playing_stat):
#     # Create a dictionary with team names as keys
#     teams = {}
#     for i in playing_stat.groupby('HomeTeam').mean().T.columns:
#         teams[i] = []
    
#     # the value corresponding to keys is a list containing the match location.
#     for i in range(len(playing_stat)):
#         HTGS = playing_stat.iloc[i]['FTHG']
#         ATGS = playing_stat.iloc[i]['FTAG']
#         teams[playing_stat.iloc[i].HomeTeam].append(HTGS)
#         teams[playing_stat.iloc[i].AwayTeam].append(ATGS)
    
#     # Create a dataframe for goals scored where rows are teams and cols are matchweek.
#     GoalsScored = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
#     GoalsScored[0] = 0
#     # Aggregate to get uptil that point
#     for i in range(2,39):
#         GoalsScored[i] = GoalsScored[i] + GoalsScored[i-1]
#     return GoalsScored


# #*** GOALS CONCEDED ****   
# # Gets the goals conceded agg arranged by teams and matchweek
# def get_goals_conceded(playing_stat):
#     # Create a dictionary with team names as keys
#     teams = {}
#     for i in playing_stat.groupby('HomeTeam').mean().T.columns:
#         teams[i] = []
    
#     # the value corresponding to keys is a list containing the match location.
#     for i in range(len(playing_stat)):
#         ATGC = playing_stat.iloc[i]['FTHG']
#         HTGC = playing_stat.iloc[i]['FTAG']
#         teams[playing_stat.iloc[i].HomeTeam].append(HTGC)
#         teams[playing_stat.iloc[i].AwayTeam].append(ATGC)
    
#     # Create a dataframe for goals scored where rows are teams and cols are matchweek.
#     GoalsConceded = pd.DataFrame(data=teams, index = [i for i in range(1,39)]).T
#     GoalsConceded[0] = 0
#     # Aggregate to get uptil that point
#     for i in range(2,39):
#         GoalsConceded[i] = GoalsConceded[i] + GoalsConceded[i-1]
#     return GoalsConceded

# def get_gss(playing_stat):
#     GC = get_goals_conceded(playing_stat)
#     GS = get_goals_scored(playing_stat)
   
#     j = 0
#     HTGS = []
#     ATGS = []
#     HTGC = []
#     ATGC = []

#     for i in range(380):
#         ht = playing_stat.iloc[i].HomeTeam
#         at = playing_stat.iloc[i].AwayTeam
#         HTGS.append(GS.loc[ht][j])
#         ATGS.append(GS.loc[at][j])
#         HTGC.append(GC.loc[ht][j])
#         ATGC.append(GC.loc[at][j])
        
#         if ((i + 1)% 10) == 0:
#             j = j + 1
        
#     playing_stat['HTGS'] = HTGS
#     playing_stat['ATGS'] = ATGS
#     playing_stat['HTGC'] = HTGC
#     playing_stat['ATGC'] = ATGC
    
#     return playing_stat

# # Apply to the filtered dataset
# for year in filtered_data:
#     year = get_gss(year)
    